In [7]:
# https://abugeau.pages.emi.u-bordeaux.fr/ai-sustainability/TP1-measurecode/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 175 kB of archives.
After this operation, 386 kB of additional disk space will be used.
Err:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 unzip amd64 6.0-26ubuntu3.2
  Could not open file /var/cache/apt/archives/partial/unzip_6.0-26ubuntu3.2_amd64.deb - open (30: Read-only file system) [IP: 185.125.190.81 80]
W: Not using locking for read only lock file /var/lib/dpkg/lock-frontend
W: Not using locking for read only lock file /var/lib/dpkg/lock
W: Not using locking for read only lock file /var/cache/apt/archives/lock
W: Problem unlinking the file /var/cache/apt/archives/partial/unzip_6.0-26ubuntu3.2_amd64.deb - PrepareFiles (30: Read-only file system)
E: Failed to fetch http://archive.ubuntu.com/ubuntu/pool/main/u/unzip/unzip

In [1]:
!lscpu

Architecture:           x86_64
  CPU op-mode(s):       32-bit, 64-bit
  Address sizes:        46 bits physical, 48 bits virtual
  Byte Order:           Little Endian
CPU(s):                 48
  On-line CPU(s) list:  0-47
Vendor ID:              GenuineIntel
  Model name:           Intel(R) Xeon(R) Gold 5118 CPU @ 2.30GHz
    CPU family:         6
    Model:              85
    Thread(s) per core: 2
    Core(s) per socket: 12
    Socket(s):          2
    Stepping:           4
    CPU max MHz:        3200.0000
    CPU min MHz:        1000.0000
    BogoMIPS:           4600.00
    Flags:              fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca
                         cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 ss ht
                         tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc art ar
                        ch_perfmon pebs bts rep_good nopl xtopology nonstop_tsc 
                        cpuid aperfmperf pni pclmulqdq dtes64 monitor ds_cpl smx
             

**Q1. What is your CPU model? The number of CPUs? The number of physical cores per CPU? The number of logical cores per CPU?**

Intel(R) Xeon(R) Gold 5118 CPU @ 2.30GHz

2 physical CPUs

12 physical cores per CPU

24 logical cores per CPU

In [2]:
!nvidia-smi -L

GPU 0: Tesla V100-PCIE-16GB (UUID: GPU-262e6c28-47c8-cffb-48c9-125a4ad20f9a)


Q2. What is the number and model of the GPUs on your machine?

One Tesla V100-PCIE-16GB

In [1]:
import zipfile
import os

# Download lab files
os.makedirs('assets/lab01', exist_ok=True)

if not os.path.exists('filesMDP.zip'):
    !wget https://www.labri.fr/perso/bugeau/Code/filesMDP.zip
    with zipfile.ZipFile('filesMDP.zip', 'r') as zip_ref:
        zip_ref.extractall('assets/lab01', pwd=b'AI&S')

--2025-09-16 14:21:10--  https://www.labri.fr/perso/bugeau/Code/filesMDP.zip
Resolving www.labri.fr (www.labri.fr)... 147.210.8.54, 2001:660:6101:404::1:80
Connecting to www.labri.fr (www.labri.fr)|147.210.8.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 420126857 (401M) [application/zip]
Saving to: 'filesMDP.zip'

filesMDP.zip        100%[===================>] 400.66M  23.5MB/s    in 28s     

2025-09-16 14:21:38 (14.5 MB/s) - 'filesMDP.zip' saved [420126857/420126857]



In [ ]:
# Q3
P_c_cpu = 105 # W; https://www.intel.com/content/www/us/en/products/sku/120473/intel-xeon-gold-5118-processor-16-5m-cache-2-30-ghz/specifications.html
P_c_gpu = 250 # W; https://images.nvidia.com/content/tesla/pdf/Tesla-V100-PCIe-Product-Brief.pdf
P_total = P_c_cpu + P_c_gpu
P_total

In [11]:
# Q4
!free -h --si
memory = 94  # In GB
P_memory = memory*0.3725  # In W
P_memory

               total        used        free      shared  buff/cache   available
Mem:             94G        5.0G        5.6G        107M         84G         88G
Swap:           4.1G        471M        3.6G


**Q5. Give some cases where real time may be greater than processing time, and vice versa.**

If there are multiple processes using the same CPU, the real time will be bigger than processing time.

If there is parallel execution (multiple cores active at the same time), processing time may be bigger than wall-clock time.

In [2]:
!ls 

DDPM_inference.py			   ddpm-butterflies-128
TrainingDDPM_one_more_epoch.py		   gpu_usage.sh
TrainingDDPM_one_more_epoch_codecarbon.py


In [2]:
# First run
!cd assets/lab01/files; python3 TrainingDDPM_one_more_epoch.py

2025-09-16 14:29:46.986524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758025787.009844 1381201 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758025787.017315 1381201 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758025787.036508 1381201 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758025787.036528 1381201 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758025787.036532 1381201 computation_placer.cc:177] computation placer alr

In [5]:
# Modify the code to measure time, run again
# Also, I was running in another process while the training was happening:
# print("psutil.cpu_percent(interval=2) = %s" % (psutil.cpu_percent(interval=2),)); print("The CPU utilization of all the CPUs is: %s" % (psutil.cpu_percent(interval=2, percpu=True), ))
!cd assets/lab01/files; python3 TrainingDDPM_one_more_epoch.py

2025-09-16 09:25:06.508605: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758007506.530974 1379191 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758007506.538199 1379191 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1758007506.556995 1379191 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758007506.557014 1379191 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1758007506.557018 1379191 computation_placer.cc:177] computation placer alr

**Q6. Run and observe the code TrainingDDPM_one_more_epoch.py. Which part of the code is measured for run time? Use the following code sample to mesure the cpu times (user time and system time).**

The total time spent executing the entirety of this software (aka wall-clock time), is given by `Real Time: 21.9420 seconds`

**Q7. What is the CPU approximate usage factor during training? You can use command htop while running to observe the usage of logical cores. Note that you can also use process.cpu_percent() to measure CPU usage.**

the utilization across all logical cores was repeatedly measured during the script execution, using `psutil.cpu_percent(interval=2, percpu=True)`. The results usually looked like:
```
[58.8, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 55.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
```
So the utilization was reasonable (50~60%) for the few cores that were in use, but the was majority of them wasn't

In [ ]:
!cd assets/lab01/files; . ./gpu_usage.sh TrainingDDPM_one_more_epoch.py

**Q8. Observe the outputs of the commands nvidia-smi and of nvidia-smi pmon. What is the mean GPU usage for the given training code when using the script?**

60.7 %


**Q9. To confirm that results are reliable, accurate, and replicable, it is necessary to repeat experiments several times—the number depending on the context. In this practice, you are asked to repeat each experiment three times and calculate the mean and variance for each. Run again the code to measure statistics of CPU and GPU usage.**

In [6]:
import numpy as np
times = [60.7, 62, 61.7, 63.3, 62.5]
print('Mean =', np.mean(times))
print('Varience =', np.var(times))

Mean = 62.04
Varience = 0.7423999999999966


**Q10. For static neural network, energy consumption is almost constant along epochs. Approximate the total energy consumption for training the model, considering 40 epochs**

In [9]:
# Define variables (example values)
run_time = 62.04*40/60  # hours
PUE = 1  # Power Usage Effectiveness
PSF = 1  # Pragmatic Scaling Factor
memory = 94  # In GB
P_memory = memory*0.3725  # In W

cores = [
    {'P_c': 105.0, 'usage_c': 0.6},  # CPU core
    {'P_c': 250, 'usage_c': 0.6204},  # GPU core
]

# Calculate total power consumption
C_total = run_time * PUE * PSF * (P_memory + sum(core['P_c'] * core['usage_c'] for core in cores))

print(f"C_total = {C_total:.2f} watt-hours")

C_total = 10468.84 watt-hours
